In [1]:
import numpy as np
import torch
from models.fft_utils import *
import matplotlib.pyplot as plt
%matplotlib inline
from data.ft_data_loader import ft_data_loader
import sys
which_dataset = 'KNEE'
testloader = ft_data_loader.get_test_loader(batch_size=64,
                                                    load_size=144,
                                                    fine_size=128,
                                                    keep_ratio=0.25,
                                                    shuffle=True,
                                                    num_workers=4,
                                                    pin_memory=True,
                                                    normalize='zero_one',
                                                    which_dataset=which_dataset)

def gen_kspace_mask(shape, ratio=0.25, mask_low_freqs = 5):
    
    h, w = shape
    mask_fft = (np.random.RandomState(42).rand(h) < ratio).astype(np.float32)
    mask_fft[:mask_low_freqs] = mask_fft[-mask_low_freqs:] = 1
    mask_fft = torch.from_numpy(mask_fft).view(1, 1, h, 1)

    return mask_fft
RFFT = RFFT().to(0)
IFFT = IFFT().to(0)
IRFFT = IRFFT().to(0)
FFT = FFT().to(0)
mask = gen_kspace_mask((128,128),ratio=0.17)
mask2 = gen_kspace_mask((128,128),ratio=0.2)
plt.imshow(mask.expand(1,1,128,128)[0,0], cmap='jet')
plt.axis('off')
iter_loader = iter(testloader)
img = iter_loader.next()[1][40:41]

ModuleNotFoundError: No module named 'models'

In [2]:
# def seperate_kspace_channel(img):
from util import util
from torchvision.utils import make_grid

kspace = RFFT(img)
k = torch.zeros(1,1,128,1)
imgs = []
wind = 32
for i in range(128//wind):
    _k = k.mul(1)
    print(range(i*wind,(i+1)*wind))
    _k[:,:,i*wind:(i+1)*wind,:] = 1
    masked_kspace = kspace * _k
    imgs.append(IFFT(masked_kspace)[:,:1,:,:])
    
I = torch.cat(imgs,0)
I2 = util.mri_denormalize(I.mul(1))
vis_img = make_grid(I2).permute(1,2,0)
plt.figure(figsize=(20,20))
plt.imshow(vis_img)

    

ModuleNotFoundError: No module named 'util'